In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import csv
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, AveragePooling2D
from sklearn.preprocessing import LabelBinarizer
from tensorflow.python.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RepeatedKFold, cross_val_score
import matplotlib.pyplot as plt

# check tensorflow gpu version
import tensorflow as tf
tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

 # check cpu e gpu
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


def open_image(X_value):
    return 1. / 255 * img_to_array(load_img('dataset/'+ X_value))


Num GPUs Available:  0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4992502288396906826
]


In [5]:
x_set = []
y_set = []

with open('index.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        if len(row) > 1:
            if row[0] and row[1] and row[2] and row[3]:
                    if os.path.isfile("dataset/" + row[0]):
                        x_set.append(row[0])
                        y_set.append(row[3])

lb = LabelBinarizer()
y_set_cat = lb.fit_transform(y_set)
X_train, X_test, y_train, y_test = train_test_split(x_set, y_set_cat, test_size=0.25, random_state=1)


NameError: name 'tf' is not defined

In [3]:
input_size = open_image(X_train[0]).shape
output_size = len(y_set_cat[0])

def baseline_model():
    model = Sequential()
    # The first two layers with 32 filters of window size 3x3
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_size))
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(output_size, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

    return model
ann_model = KerasClassifier(build_fn=baseline_model, epochs=50, batch_size=8)
fin_model = KerasClassifier(build_fn=baseline_model, epochs=50, batch_size=8)
X_train_adapt = np.array(list(map(open_image, X_train)))
baseline_model().summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 32)      896       
_________________________________________________________________
average_pooling2d (AveragePo (None, 128, 128, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      18496     
_________________________________________________________________
average_pooling2d_1 (Average (None, 64, 64, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 128)       7

In [4]:
kfold= RepeatedKFold(n_splits=5, n_repeats=1)
results= cross_val_score(ann_model, X_train_adapt, y_train, cv=kfold, n_jobs=4, verbose=1)  # 4 cpus
results.mean()  # Mean MSE

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    5.4s remaining:    8.1s


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {SIGKILL(-9)}

In [ ]:

history = fin_model.fit(X_train_adapt, y_train)

In [ ]:
plt.plot(history.history['acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()